# Input annotations into a PostgreSQL database 

Requirements: 
- download and install [PostgreSQL](https://www.postgresql.org/download/windows/)
- download and install [pgAdmin 4](https://www.pgadmin.org/download/)

### 1) Create database 
In pgAdmin, create a database to store the annotations 


### 2) Create a .env file 
For the dtpdb library to be able to access the database we just created, create a .env file that contains the access information. 

- Open a text file and save it as ".env"
- The text file should contain:  
```
DATABASE_URL = "dbname=YOUR_DB_NAME user=YOUR_USERNAME(usually postgres) password=YOUR_PASSWORD" 
```


## Install and import libraries

In [1]:
!pip install -q pip --upgrade

!pip install -q Pillow==7.2.0
!pip install -q psycopg2-binary==2.8.6
!pip install -q python-dotenv==0.14.0

In [2]:
import json

from dtpdb import input_data

### 3) For a new database
Create new tables and input new data:  
`create_main_tables()` Creates **Images**, **Annotations** and **Categories** tables in the database 

add data to tables: 
- `add_image_data(data)` Adds image data 
- `add_category_data(data)` Adds category data
- `add_annotation_data(data)` Adds annotation data

In [7]:
input_data.create_main_tables()

In [8]:
with open('raw data\coco_test_file.json') as json_file:
    data = json.load(json_file)
    
input_data.add_image_data(data)
input_data.add_category_data(data)
input_data.add_annotation_data(data)

### 4) To add data to an existing database
1) Create temp tables to store the data temporarily using `create_temp_tables()`  

2) Add data from new annotations file:
- `add_image_data_temp(data)` Adds image data 
- `add_category_data_temp(data)` Adds category data
- `add_annotation_data_temp(data)` Adds annotation data  

3) Get the largest id in the main images table, then update temp images table to increment from that value. The image_id in the annotations table will also update accordingly. 
`reorder_id()`  

4) Update data in main tables. 
- `update_images()`
- `update_categories()`
- `update_annotations()`

The function also checks the image name in the temp table with the image name in the main table and does not add the temp image if the same name exists in the main table. 

5) Delete the temp tables 
`delete_temp_tables()`


In [10]:
input_data.create_temp_tables()

In [12]:
# change the file name/file path to the annotations file we are using as update
with open("raw data//via_project_30Sep2020_11h56m_coco.json") as json_file:
    data = json.load(json_file)

# add data from file to temp tables
input_data.add_image_data_temp(data)
input_data.add_category_data_temp(data)
input_data.add_annotations_data_temp(data)

# reorder temp images id to start with max id in main images table
input_data.reorder_id()

# update data in main table
input_data.update_images()
input_data.update_categories()
input_data.update_annotations()

# Delete temp tables
input_data.delete_temp_tables()